In [1]:
import pandas as pd

interjection_df = pd.read_csv("interjections.csv")
interjection_df = interjection_df[interjection_df.columns[1:]]
#!ls ../../../../../../Replication-file-Robust-Text-Analysis/

separation_df = pd.read_excel("../../../../../../Replication-file-Robust-Text-Analysis/Separation.xlsx")

meeting_df = pd.read_csv("../../../derivation/python/output/meeting_derived_file.csv")



In [27]:
print(separation_df[separation_df.Date.dt.year==1992])

         date_string  FOMC1_start  FOMC1_end  FOMC2_start FOMC2_end       Date
date_ind                                                                      
199202        199202           71        151          310       367 1992-02-01
199203        199203           78        146          147       215 1992-03-01
199205        199205           37        163          136       270 1992-05-01
199207        199207           48        156          367       535 1992-07-01
199208        199208           42        133          134       223 1992-08-01
199210        199210           81        229           86       305 1992-10-01
199211        199211           29         88           89       280 1992-11-01
199212        199212           19        129          130       206 1992-12-01


In [2]:
separation_df = separation_df.rename(columns={separation_df.columns[0]:"date_string"})
separation_df.date_string = separation_df.date_string.apply(str)

separation_df['Date'] = pd.to_datetime(separation_df.date_string,format="%Y%m")
#print(separation_df)

interjection_df['Date'] = pd.to_datetime(interjection_df.Date)
interjection_df = interjection_df[(interjection_df.Date>pd.to_datetime("1987-07-31"))&
                                  (interjection_df.Date<pd.to_datetime("2006-02-01"))]


interjection_df['date_string'] = interjection_df.Date.apply(lambda x: x.strftime("%Y%m")).apply(str)
print(len(set(interjection_df.date_string)))
print(len(set(separation_df.date_string)))

print(separation_df.date_string)

170
148
0      198708
1      198709
2      198711
3      198712
4      198802
5      198803
6      198806
7      198808
8      198809
9      198811
10     198812
11     198902
12     198903
13     198905
14     198907
15     198908
16     198910
17     198911
18     198912
19     199002
20     199003
21     199005
22     199007
23     199008
24     199010
25     199011
26     199012
27     199102
28     199103
29     199105
        ...  
118    200206
119    200208
120    200209
121    200211
122    200212
123    200301
124    200303
125    200305
126    200306
127    200308
128    200309
129    200310
130    200312
131    200401
132    200403
133    200405
134    200406
135    200408
136    200409
137    200411
138    200412
139    200502
140    200503
141    200505
142    200506
143    200508
144    200509
145    200511
146    200512
147    200601
Name: date_string, Length: 148, dtype: object


In [3]:
cc_df = meeting_df[meeting_df.event_type=="Conference Call"]
cc_df['Date'] = pd.to_datetime(cc_df['start_date'])

interjection_df = interjection_df[~interjection_df['Date'].isin(cc_df['Date'])]

print(len(set(interjection_df.date_string)))

149


/home/anand/Code/columbia/venv/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [6]:
print(len(set(separation_df.date_string)))
print(len(set(interjection_df.date_string)))


separation_dates = sorted(list(set(separation_df.date_string)))

interjection_dates = sorted(list(set(interjection_df.date_string)))

missing = set(separation_dates)-set(interjection_dates)
print(missing)
pass
'''
for s_d,i_d in zip(separation_dates,interjection_dates):
    print("Separation:",s_d)
    print("---")
    print("Interjection",i_d)
    print("==="*20)
'''

separation_df['date_ind'] = separation_df.date_string.astype(int)
separation_df = separation_df.set_index('date_ind')

148
149
{'199207', '199502', '199807'}


In [23]:
meeting_groups = interjection_df.groupby("Date")
final_df = pd.DataFrame(columns=interjection_df.columns)
for date_ind in interjection_df['date_string'].astype(int):
    meeting_date = interjection_df[interjection_df.date_string.astype(int)==date_ind].reset_index(drop=True)
    meeting_date['FOMC_Section'] = 0
    if date_ind not in list(separation_df.index):
        final_df = pd.concat([final_df, meeting_date], ignore_index = True)
        continue
    
    meeting_date.loc[separation_df['FOMC1_start'][date_ind]:
                              separation_df['FOMC1_end'][date_ind],"FOMC_Section"] = 1
    #print(FOMC1)
    if separation_df['FOMC2_end'][date_ind] == 'end':
        meeting_date.loc[separation_df['FOMC2_start'][date_ind]:
                          ,"FOMC_Section"] = 2
    else:
        meeting_date.loc[separation_df['FOMC2_start'][date_ind]:
                          separation_df['FOMC2_end'][date_ind],"FOMC_Section"]=2
    #FOMC2 = meeting_date.iloc[separation['FOMC2_start'][date]:]

    final_df = pd.concat([final_df, meeting_date], ignore_index = True)

final_df

/home/anand/Code/columbia/venv/lib/python3.6/site-packages/ipykernel_launcher.py:21: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/home/anand/Code/columbia/venv/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


KeyboardInterrupt: 